In [1]:
#establishing environment
import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd
import pickle
import scipy
from scipy.stats import pearsonr, ttest_rel, ttest_ind
from scipy import signal as sig
import matplotlib.patches as mpatches
import mat73
from scipy.io import loadmat, savemat
import mne
#conda install git+https://github.com/ieeg-portal/ieegpy.git # Install ieegpy toolbox directly from github
from ieeg.auth import Session
import random
from sklearn.cluster import KMeans
import os
#os.chdir('E:/CNT/Interictal_Spike_Analysis/HUMAN/working_feat_extract_code/functions/')
os.chdir('/Users/carlosaguila/PycharmProjects/CNT_Interictal_Spikes/Interictal_Spike_Analysis/HUMAN/working_feat_extract_code/functions')
from ied_functions import *
import warnings
from ieegrecon_labels import *
warnings.filterwarnings('ignore')

In [13]:
json_labels = '/Users/carlosaguila/PycharmProjects/CNT_Interictal_Spikes/Patient/pt_database/CNT_iEEG_BIDS/sub-RID0179/derivatives/ieeg_recon/module3/sub-RID0179_ses-research3T_space-T00mri_atlas-DKTantspynet_radius-2_desc-vox_coordinates.json'

In [14]:
workinglabels = pd.read_json(json_labels, lines=True)

In [15]:
empty = (workinglabels[workinglabels['label'] == 'EmptyLabel'])
display(empty)

,name,x,y,z,index,label,labels_sorted,percent_assigned
2,LAT3,66.5970,137.4260,26.3088,0,EmptyLabel,"[EmptyLabel, left inferior temporal ]","[0.7272727273, 0.2727272727]"
3,LAT4,57.2303,134.0920,30.9649,0,EmptyLabel,"[EmptyLabel, left inferior temporal ]","[0.6666666667000001, 0.33333333330000003]"
7,LDA4,64.2987,136.7100,40.5375,0,EmptyLabel,"[EmptyLabel, left fusiform ]","[0.9696969697000001, 0.030303030300000002]"
8,LDH1,74.5979,126.0560,44.7521,0,EmptyLabel,"[EmptyLabel, left parahippocampal ]","[0.7575757576000001, 0.24242424240000002]"
10,LDH3,64.1822,125.8690,43.9083,0,EmptyLabel,"[EmptyLabel, left fusiform ]","[0.6060606061, 0.3939393939]"
11,LDH4,58.7263,125.7710,43.4664,0,EmptyLabel,"[EmptyLabel, left fusiform , left inferior t...","[0.3636363636, 0.33333333330000003, 0.303030303]"
12,LPT1,67.1344,106.0680,48.2641,0,EmptyLabel,"[EmptyLabel, left fusiform ]","[0.7272727273, 0.2727272727]"
14,LPT3,54.8546,118.4140,38.0928,0,EmptyLabel,"[EmptyLabel, left inferior temporal ]","[0.5757575758, 0.4242424242]"
16,RDA1,132.1970,145.9820,45.0470,0,EmptyLabel,[EmptyLabel],[1.0]
17,RDA2,134.7750,141.4250,45.0775,0,EmptyLabel,[EmptyLabel],[1.0]


In [16]:
def unnesting(df, explode, axis):
    if axis==1:
        idx = df.index.repeat(df[explode[0]].str.len())
        df1 = pd.concat([
            pd.DataFrame({x: np.concatenate(df[x].values)}) for x in explode], axis=1)
        df1.index = idx

        return df1.join(df.drop(explode, 1), how='right')
    else :
        df1 = pd.concat([
                         pd.DataFrame(df[x].tolist(), index=df.index).add_prefix(x) for x in explode], axis=1)
        return df1.join(df.drop(explode, 1), how='right')

In [17]:
empty = unnesting(empty, ['labels_sorted', 'percent_assigned'], axis=0)
display(empty)

,labels_sorted0,labels_sorted1,labels_sorted2,percent_assigned0,percent_assigned1,percent_assigned2,name,x,y,z,index,label
2,EmptyLabel,left inferior temporal,None,0.727273,0.272727,NaN,LAT3,66.5970,137.4260,26.3088,0,EmptyLabel
3,EmptyLabel,left inferior temporal,None,0.666667,0.333333,NaN,LAT4,57.2303,134.0920,30.9649,0,EmptyLabel
7,EmptyLabel,left fusiform,None,0.969697,0.030303,NaN,LDA4,64.2987,136.7100,40.5375,0,EmptyLabel
8,EmptyLabel,left parahippocampal,None,0.757576,0.242424,NaN,LDH1,74.5979,126.0560,44.7521,0,EmptyLabel
10,EmptyLabel,left fusiform,None,0.606061,0.393939,NaN,LDH3,64.1822,125.8690,43.9083,0,EmptyLabel
11,EmptyLabel,left fusiform,left inferior temporal,0.363636,0.333333,0.303030,LDH4,58.7263,125.7710,43.4664,0,EmptyLabel
12,EmptyLabel,left fusiform,None,0.727273,0.272727,NaN,LPT1,67.1344,106.0680,48.2641,0,EmptyLabel
14,EmptyLabel,left inferior temporal,None,0.575758,0.424242,NaN,LPT3,54.8546,118.4140,38.0928,0,EmptyLabel
16,EmptyLabel,None,None,1.000000,NaN,NaN,RDA1,132.1970,145.9820,45.0470,0,EmptyLabel
17,EmptyLabel,None,None,1.000000,NaN,NaN,RDA2,134.7750,141.4250,45.0775,0,EmptyLabel


In [19]:
empty = empty[np.isnan(empty['percent_assigned1']) == False]

In [20]:
display(empty)

,labels_sorted0,labels_sorted1,labels_sorted2,percent_assigned0,percent_assigned1,percent_assigned2,name,x,y,z,index,label
2,EmptyLabel,left inferior temporal,None,0.727273,0.272727,NaN,LAT3,66.5970,137.4260,26.3088,0,EmptyLabel
3,EmptyLabel,left inferior temporal,None,0.666667,0.333333,NaN,LAT4,57.2303,134.0920,30.9649,0,EmptyLabel
7,EmptyLabel,left fusiform,None,0.969697,0.030303,NaN,LDA4,64.2987,136.7100,40.5375,0,EmptyLabel
8,EmptyLabel,left parahippocampal,None,0.757576,0.242424,NaN,LDH1,74.5979,126.0560,44.7521,0,EmptyLabel
10,EmptyLabel,left fusiform,None,0.606061,0.393939,NaN,LDH3,64.1822,125.8690,43.9083,0,EmptyLabel
11,EmptyLabel,left fusiform,left inferior temporal,0.363636,0.333333,0.303030,LDH4,58.7263,125.7710,43.4664,0,EmptyLabel
12,EmptyLabel,left fusiform,None,0.727273,0.272727,NaN,LPT1,67.1344,106.0680,48.2641,0,EmptyLabel
14,EmptyLabel,left inferior temporal,None,0.575758,0.424242,NaN,LPT3,54.8546,118.4140,38.0928,0,EmptyLabel
18,EmptyLabel,right middle temporal,right superior temporal,0.909091,0.060606,0.030303,RDA3,137.7450,137.0660,46.2329,0,EmptyLabel
19,EmptyLabel,right middle temporal,None,0.969697,0.030303,NaN,RDA4,140.7160,132.7070,47.3884,0,EmptyLabel


In [21]:
changed = empty[empty['percent_assigned1'] > 0.25]


In [22]:
display(changed)

,labels_sorted0,labels_sorted1,labels_sorted2,percent_assigned0,percent_assigned1,percent_assigned2,name,x,y,z,index,label
2,EmptyLabel,left inferior temporal,None,0.727273,0.272727,NaN,LAT3,66.5970,137.4260,26.3088,0,EmptyLabel
3,EmptyLabel,left inferior temporal,None,0.666667,0.333333,NaN,LAT4,57.2303,134.0920,30.9649,0,EmptyLabel
10,EmptyLabel,left fusiform,None,0.606061,0.393939,NaN,LDH3,64.1822,125.8690,43.9083,0,EmptyLabel
11,EmptyLabel,left fusiform,left inferior temporal,0.363636,0.333333,0.303030,LDH4,58.7263,125.7710,43.4664,0,EmptyLabel
12,EmptyLabel,left fusiform,None,0.727273,0.272727,NaN,LPT1,67.1344,106.0680,48.2641,0,EmptyLabel
14,EmptyLabel,left inferior temporal,None,0.575758,0.424242,NaN,LPT3,54.8546,118.4140,38.0928,0,EmptyLabel
21,EmptyLabel,right fusiform,None,0.696970,0.303030,NaN,RDH2,131.6930,120.7490,44.5716,0,EmptyLabel
26,EmptyLabel,right rostral middle frontal,right pars triangularis,0.666667,0.272727,0.060606,RMF3,141.6660,175.9700,82.9094,0,EmptyLabel
28,EmptyLabel,right rostral middle frontal,right pars triangularis,0.575758,0.363636,0.060606,RMF5,139.7020,160.4910,96.3756,0,EmptyLabel
31,EmptyLabel,right superior temporal,right middle temporal,0.424242,0.393939,0.181818,RP2,149.9140,84.7203,70.8063,0,EmptyLabel


In [45]:
data_directory = '/Users/carlosaguila/PycharmProjects/CNT_Interictal_Spikes/Patient/pt_database'
ptnames = ['HUP105','HUP106','HUP107','HUP111']
spike, brain_df, ids = load_ptall(ptnames[3],data_directory)


In [48]:
pd.set_option('display.max_rows', 200)
brain_df['name'] = brain_df['name'].str.replace('-CAR','')
display(brain_df.merge(changed[['labels_sorted1', 'percent_assigned1']], left_on=brain_df['name'], right_on=changed['name'], how = 'left', indicator=True))

,key_0,Unnamed: 0,name,x,y,z,index,label,labels_sorted1,percent_assigned1,_merge
0,LAT1,0,LAT1,80.8064,149.0920,32.5711,1009.0,left inferior temporal,NaN,NaN,left_only
1,LAT2,1,LAT2,74.9307,142.1780,27.1623,1009.0,left inferior temporal,NaN,NaN,left_only
2,LAT3,2,LAT3,66.5970,137.4260,26.3088,0.0,EmptyLabel,left inferior temporal,0.272727,both
3,LAT4,3,LAT4,57.2303,134.0920,30.9649,0.0,EmptyLabel,left inferior temporal,0.333333,both
4,LDA1,4,LDA1,79.6568,139.6700,41.4514,1006.0,left entorhinal,NaN,NaN,left_only
5,LDA2,5,LDA2,74.7373,138.6230,40.7986,1006.0,left entorhinal,NaN,NaN,left_only
6,LDA3,6,LDA3,69.2384,137.2780,41.0649,1006.0,left entorhinal,NaN,NaN,left_only
7,LDA4,7,LDA4,64.2987,136.7100,40.5375,0.0,EmptyLabel,NaN,NaN,left_only
8,LDH1,8,LDH1,74.5979,126.0560,44.7521,0.0,EmptyLabel,NaN,NaN,left_only
9,LDH2,9,LDH2,69.6380,125.9670,44.3503,1016.0,left parahippocampal,NaN,NaN,left_only


In [46]:
print(ids)

['HUP111', 'sub-RID0179']


In [ ]:
def unnesting(df, explode, axis):
    if axis==1:
        idx = df.index.repeat(df[explode[0]].str.len())
        df1 = pd.concat([
            pd.DataFrame({x: np.concatenate(df[x].values)}) for x in explode], axis=1)
        df1.index = idx

        return df1.join(df.drop(explode, 1), how='right')
    else :
        df1 = pd.concat([
                         pd.DataFrame(df[x].tolist(), index=df.index).add_prefix(x) for x in explode], axis=1)
        return df1.join(df.drop(explode, 1), how='right')

def load_rid_forjson(ptname, data_directory):
    ptids = pd.read_csv(data_directory + '/pt_data/all_ptids.csv')
    rid = ptids['r_id'].loc[ptids['hup_id'] == ptname].astype('string')
    rid = np.array(rid)
    dkt_directory = data_directory + 'CNT_iEEG_BIDS/{}/derivatives/ieeg_recon/module3/{}_ses-research3T_space-T00mri_atlas-DKTantspynet_radius-2_desc-vox_coordinates.csv'.format(rid[0],rid[0])
    brain_df = pd.read_csv(dkt_directory)
    brain_df['name'] = brain_df['name'].astype(str) + '-CAR'
    return rid[0], brain_df
        
def label_fix(pt, data_directory, threshold = 0.25):
    '''
    label_fix reassigns labels overlapping brain regions to "empty labels" in our DKTantspynet output from IEEG_recon
    input:  pt - name of patient. example: 'HUP100' 
            data_directory - directory containing CNT_iEEG_BIGS folder. (must end in '/')
            threshold - arbitrary threshold that r=2mm surround of electrode must overlap with a brain region. default: threshold = 25%, Brain region has a 25% or more overlap.
    output: relabeled_df - a dataframe that contains 2 extra columns showing the second most overlapping brain region and the percent of overlap. 
    '''

    rid, brain_df = load_rid_forjson(pt, data_directory)
    json_labels = data_directory + 'CNT_iEEG_BIDS/{}}/derivatives/ieeg_recon/module3/{}}_ses-research3T_space-T00mri_atlas-DKTantspynet_radius-2_desc-vox_coordinates.json'.format(rid,rid)
    workinglabels = pd.read_json(json_labels, lines=True)
    empty = (workinglabels[workinglabels['label'] == 'EmptyLabel'])
    empty = unnesting(empty, ['labels_sorted', 'percent_assigned'], axis=0)
    empty = empty[np.isnan(empty['percent_assigned1']) == False]
    changed = empty[empty['percent_assigned1'] >= threshold]
    
    brain_df['name'] = brain_df['name'].str.replace('-CAR','')
    relabeled_df = brain_df.merge(changed[['labels_sorted1', 'percent_assigned1']], left_on=brain_df['name'], right_on=changed['name'], how = 'left', indicator=True)

    return relabeled_df